## Install packages:
1. beautySoup4 to scrape Wiki page
pip install beautysoup5

2. The xml parser to parse page:
pip install lxmp

pip install html5lib

pip install requests

First import all libraries.

In [207]:
from bs4 import BeautifulSoup
import requests
import lxml.html as lh
import pandas as pd

In [208]:
# 1. First, I assign the link of the website through 
# which we are going to scrape the data and assign 
# it to variable named website_url
url = 'http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

# 2. Next, I creat a BeautifulSoup (soup) object
page = requests.get(url)

#Store the contents of the website under doc
doc = lh.fromstring(page.content)


#Store the contents of the website under doc
# After  carefully inspect the HTML script all the table 
# contents is under class Wikitable Sortable. 
# So first task is to find class ‘wikitable sortable’ in the HTML script.
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

#Check the length of the first 12 rows
[len(T) for T in tr_elements[:12]]


[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]


Looks like all our rows have exactly 3 columns.

Next, let’s parse the first row as our header.

In [209]:
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print '%d:"%s"'%(i,name)
    col.append((name,[]))

1:"Postcode"
2:"Borough"
3:"Neighbourhood
"


Creating Pandas DataFrame
Each header is appended to a tuple along with an empty list.

In [210]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

Just to be sure, let’s check the length of each column. Ideally, they should all be the same.

In [211]:
[len(C) for (title,C) in col]

[289, 289, 289]

CREATING  a DataFrame

In [212]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)
df.columns=['Borough',"Neighbourhood","Postcode"]
print(df.head())
print(df.shape)

            Borough       Neighbourhood Postcode
0      Not assigned      Not assigned\n      M1A
1      Not assigned      Not assigned\n      M2A
2        North York         Parkwoods\n      M3A
3        North York  Victoria Village\n      M4A
4  Downtown Toronto      Harbourfront\n      M5A
(289, 3)


## Cleaning our data

In [213]:
# 1. removing all occurence of  \n
df=df.replace({'\n': ''}, regex=True)
print(df.head())

            Borough     Neighbourhood Postcode
0      Not assigned      Not assigned      M1A
1      Not assigned      Not assigned      M2A
2        North York         Parkwoods      M3A
3        North York  Victoria Village      M4A
4  Downtown Toronto      Harbourfront      M5A


In [214]:
#2. Dropping  rows with a borough that is Not assigned.
print(df[df['Borough']=="Not assigned"].shape)
df=df[(df['Borough']!="Not assigned")]
df.head()

(77, 3)


,Borough,Neighbourhood,Postcode
2,North York,Parkwoods,M3A
3,North York,Victoria Village,M4A
4,Downtown Toronto,Harbourfront,M5A
5,Downtown Toronto,Regent Park,M5A
6,North York,Lawrence Heights,M6A


We have 77 rows with Borough that is Not assigned.

In [215]:
# Check unique values in each column:
print(len(df.Borough.unique()))
print(len(df.Postcode.unique()))
print(len(df.Neighbourhood.unique()))

11
103
210


We have 11 unique Borough, 103 unique Postcodes, 209 unique Neighbourhood. 


## Processing Data Frame
#4. Combining multiple neighborhood values into one rows for one postal code area.

In [216]:
df_test = df
df_g=df_test.groupby('Postcode')['Neighbourhood'].apply(','.join).reset_index()
df_g


,Postcode,Neighbourhood
0,M1B,"Rouge,Malvern"
1,M1C,"Highland Creek,Rouge Hill,Port Union"
2,M1E,"Guildwood,Morningside,West Hill"
3,M1G,Woburn
4,M1H,Cedarbrae
5,M1J,Scarborough Village
6,M1K,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,"Clairlea,Golden Mile,Oakridge"
8,M1M,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,"Birch Cliff,Cliffside West"


In [217]:
df_part2=df[["Borough","Postcode"]] 
df_part2.head()

,Borough,Postcode
2,North York,M3A
3,North York,M4A
4,Downtown Toronto,M5A
5,Downtown Toronto,M5A
6,North York,M6A


In [218]:
#Joining two dataframes 
df_res=df_g.join(df_part2.set_index('Postcode'), on='Postcode')


In [221]:
#removing duplicates:
df_res_unique = df_res.drop_duplicates(subset=['Postcode', 'Neighbourhood','Borough'], keep=False)
df_res_unique.shape

(45, 3)

In [222]:
df_test = df_res_unique

In [223]:
df_test[df_test['Neighbourhood']=="Not assigned"]

,Postcode,Neighbourhood,Borough
85,M7A,Not assigned,Queen's Park


In [224]:
#only one value, easely replace it
df_test=df_test.set_value(85, 'Neighbourhood', 'Queen\'s Park')

In [225]:
df_test.head(10)

,Postcode,Neighbourhood,Borough
3,M1G,Woburn,Scarborough
4,M1H,Cedarbrae,Scarborough
5,M1J,Scarborough Village,Scarborough
12,M1S,Agincourt,Scarborough
16,M1X,Upper Rouge,Scarborough
17,M2H,Hillcrest Village,North York
19,M2K,Bayview Village,North York
22,M2N,Willowdale South,North York
23,M2P,York Mills West,North York
24,M2R,Willowdale West,North York


## Saving our results

In [227]:
df_test.to_csv("Toronto_part1.csv")

In [226]:

#In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.
print(df_test.shape)

(45, 3)
